This notebook shows a temporary fix to make the outputs of the FF SDK align to the file type that BP3D expects

In [ ]:
# import relevant libraries
from scipy.io import FortranFile 
import numpy as np
import zarr
import json

import copy
import os


In [ ]:
# functions
def read_dat_file(filename, nz, ny, nx, order= "C"):
    """
    Read in a .dat file as a numpy array.

    Parameters
    ----------
    filename : Path or str
        The path to the .dat file to read.
    nz : int
        The number of cells in the z-direction.
    ny : int
        The number of cells in the y-direction.
    nx : int
        The number of cells in the x-direction.
    order : str, optional
        The order of the array. Default is "C".

    Returns
    -------
    ndarray
        A 3D numpy array representing the data in the .dat file. The array
        has dimensions (nz, ny, nx).
    """
    if isinstance(filename, str):
        filename = (filename)

    with open(filename, "rb") as fin:
        arr = (
            FortranFile(fin)
            .read_reals(dtype="float32")
            .reshape((nz, ny, nx), order=order)
        )

    return arr

def _write_np_array_to_dat(array, dat_name,
                           output_dir, dtype = np.float32):
    """
    Write a numpy array to a fortran binary file. Array must be cast to the
    appropriate data type before calling this function. If the array is 3D,
    the array will be reshaped from (y, x, z) to (z, y, x) for fortran.
    """
    # # Reshape array from (y, x, z) to (z, y, x) (also for fortran)
    # if len(array.shape) == 3:
    #     array = np.moveaxis(array, 2, 0).astype(dtype)
    # else:
    #     array = array.astype(dtype)

    # Write the zarr array to a dat file with scipy FortranFile package
    with FortranFile(output_dir + dat_name, "w") as f:
        f.write_record(array)

def edit_dat(array_sz, edited_ff_path, gui_dir, file, zarrname, fuel_moisture = None):
    '''
    Edit and write .dat files for different surface/canopy variables.

    Parameters
    ----------
    array_sz : dict
        Dictionary with keys 'y', 'x', 'z' specifying array dimensions.
    edited_ff_path : str or Path
        Path to the edited zarr file.
    gui_dir : str
        Directory to write the output .dat file.
    file : str
        Output .dat filename.
    zarrname : str
        Name of the variable to extract from the zarr file.
    fuel_moisture : float or None, optional
        Value to set for fuel moisture if zarrname == 'fuelMoisture'.
    '''
    edited_ff = zarr.open(edited_ff_path)
    # Initialize a blank array with shape (y, x, z)
    arr = np.zeros((array_sz['y'], array_sz['x'], array_sz['z']), dtype = 'float32')
    
    # Handle different variable names and extract/transform data accordingly
    if zarrname == 'SAVR':
        # Extract 'SAVR' from surface/oneHour, invert values, and assign to arr[:,:,0]
        zarrarray_surface = (edited_ff['surface'][zarrname]['oneHour'][...])
        arr[:,:,0] = zarrarray_surface[:array_sz['y'], :array_sz['x']]
        arr[arr != 0] = 2/arr[arr != 0]
        arr[:,:,0] = 2/zarrarray_surface[:array_sz['y'], :array_sz['x']]

    if zarrname == 'fuelDepth':
        # Extract 'fuelDepth' from surface and assign to arr[:,:,0]
        zarrarray_surface = (edited_ff['surface'][zarrname][...])
        arr[:,:,0] = zarrarray_surface[:array_sz['y'], :array_sz['x']]
    
    if zarrname == 'fuelMoisture':
        # Set all values in arr[:,:,0] to the provided fuel_moisture value
        zarrarray_surface = (edited_ff['surface'][zarrname][...])
        arr[:,:,0] = zarrarray_surface[:array_sz['y'], :array_sz['x']]
        arr[:,:,0] = fuel_moisture
    
    # Replace NaNs with zeros
    arr[np.isnan(arr)] = 0

    # Change array shape from (y, x, z) to (z, y, x) for Fortran compatibility
    arr = np.rollaxis(arr, 2)
    
    if zarrname == 'fuelLoad':
        # For 'fuelLoad', use tree bulkDensity for canopy, and surface oneHour for surface
        arr = (edited_ff['tree']['bulkDensity'][...])
        zarrarray_surface = (edited_ff['surface'][zarrname]['oneHour'][...])
        arr[0,:,:] = zarrarray_surface[:array_sz['y'], :array_sz['x']]
    
    # Flip each z-slice to match required orientation
    arr_flipped = np.zeros((array_sz['z'], array_sz['y'], array_sz['x']), dtype = 'float32')
    arr_flipped[0, :,:] = np.flipud(np.fliplr(arr[0,:,:]))

    for zi in np.arange(1, array_sz['z']):
        arr_flipped[zi, :,:] = np.flipud(np.fliplr(arr[zi,:,:]))

    # Write the processed array to a Fortran binary .dat file
    _write_np_array_to_dat(arr_flipped, file, gui_dir)

def edit_topo_dat(array_sz, edited_ff_path, gui_dir, file):
    '''
    Edit and write the topography .dat file (e.g., fastfuels_topo.dat).

    Parameters
    ----------
    array_sz : dict
        Dictionary with keys 'y', 'x', 'z' specifying array dimensions.
        For topography, z should be 1.
    edited_ff_path : str or Path
        Path to the edited fast fuels compressed zarr file.
    gui_dir : str
        Directory to write the output .dat file.
    file : str
        Output .dat filename.
    '''
    # Open the zarr file containing the topography data
    edited_ff = zarr.open(edited_ff_path)
    
    # Initialize a blank array with shape (1, y, x) for topography (z=1)
    arr = np.zeros((1, array_sz['y'], array_sz['x']), dtype='float32')

    # Read the elevation data from the zarr file and assign to arr[0, :, :]
    zarrarray_surface = edited_ff['topography']['elevation'][...]
    arr[0, :, :] = zarrarray_surface[:array_sz['y'], :array_sz['x']]

    # Flip the array vertically to match required orientation for output
    arr_flipped = np.zeros((1, array_sz['y'], array_sz['x']), dtype='float32')
    arr_flipped[0, :, :] = np.flipud(arr[0, :, :])
    
    # Write the processed array to a Fortran binary .dat file
    _write_np_array_to_dat(arr_flipped, file, gui_dir)

# Transpose the array from (z, y, x) to (y, x, z)
def make_shape(arr):
    """
    Transpose a 3D numpy array from (z, y, x) to (y, x, z).
    arr : Input array with shape (z, y, x).
    Returns a transposed array with shape (y, x, z).
    """
    return np.transpose(arr, (1, 2, 0))

def make_canopy_arr(arr):
    """
    Set the first z-slice of a 3D array to zero for the canopy.
    arr is a 3D numpy array.

    Returns a copy of arr with zeroed first z-slice (zeroarr[:,:,0] = 0).
    """
    zeroarr = copy.deepcopy(arr)
    zeroarr[:,:,0] = 0
    return zeroarr


In [ ]:
# Set the directory containing the GUI files
gui_dir = '/path/to/QF/GUI/directory/'
zipname = 'zipname.zip' 
# Path to the edited fast fuels zarr file
edited_ff_path = gui_dir + zipname

# Open the compressed zarr file saved from FF SDK
z = zarr.open(edited_ff_path)

# Start building the attribute files to add into the updated fuel-array.zip
nx = dict(z.attrs)['nx']
ny = dict(z.attrs)['ny']
nz = z['tree']['bulkDensity'].shape[0]

# Calculate spatial extents using affine transformation from attributes
xmin = (dict(z.attrs)['affine'][2] + 1) # x minimum coordinate
ymax = (dict(z.attrs)['affine'][5] + 1) # y maximum coordinate
xmax = xmin + (nx * 2) - 2             # x maximum coordinate
ymin = ymax - (ny * 2) + 2             # y minimum coordinate

# Read in GUI data from JSON file
f = open(gui_dir + 'GUIdata.json',) # GUIdata.json
guidata = json.load(f)

In [ ]:
# Set the desired fuel moisture value
fuel_moisture = 0.2 # what your ideal fuel moisture is

# Define the array size dictionary using nx, ny, nz from above
array_sz = {}
array_sz['x'] = nx
array_sz['y'] = ny
array_sz['z'] = nz

# Load GUI data from the original fuels directory
f = open(gui_dir + 'orig_fuels/GUIdata.json',)
guidata = json.load(f)

# Edit and write the .dat files from fastfuels for each variable
edit_dat(array_sz, edited_ff_path, gui_dir, 'treesss.dat', 'SAVR')
edit_dat(array_sz, edited_ff_path, gui_dir, 'treesfueldepth.dat',  'fuelDepth')
edit_dat(array_sz, edited_ff_path, gui_dir, 'treesmoist.dat', 'fuelMoisture', fuel_moisture=fuel_moisture)
edit_dat(array_sz, edited_ff_path, gui_dir, 'treesrhof.dat',  'fuelLoad')
edit_topo_dat(array_sz, edited_ff_path, gui_dir, 'fastfuels_topo.dat')


/var/folders/rp/zdr20wpx60j7227xn_x7mjqc0000gn/T/ipykernel_2094/1398222598.py:75: RuntimeWarning: divide by zero encountered in divide
  arr[:,:,0] = 2/zarrarray_surface[:array_sz['y'], :array_sz['x']]


In [ ]:
# Rebuild the fuel-array.zip to match the previous version of fastfuels with the new dat files

# Read and reshape the SAVR data from the .dat file
sav = read_dat_file(gui_dir + 'treesss.dat', nz, ny, nx, order="C")
sav = make_shape(sav)

# Read and reshape the fuel depth data from the .dat file
fueldepth = read_dat_file(gui_dir + 'treesfueldepth.dat', nz, ny, nx, order="C")
fueldepth = make_shape(fueldepth)

# Read and reshape the fuel moisture content data from the .dat file
fmc = read_dat_file(gui_dir + 'treesmoist.dat', nz, ny, nx, order="C")
fmc = make_shape(fmc)

# Read and reshape the bulk density data from the .dat file
bulkdensity = read_dat_file(gui_dir + 'treesrhof.dat', nz, ny, nx, order="C")
bulkdensity = make_shape(bulkdensity)

# Read and reshape the DEM (topography) data from the .dat file
dem = read_dat_file(gui_dir + 'fastfuels_topo.dat', 1, ny, nx, order="C")
dem = make_shape(dem)

# Define the shapes for canopy and surface arrays
canopy_shape = (ny, nx, nz)
surface_shape = (ny, nx)


# Extract the species code array from the zarr file
speciescode = z['tree']['SPCD'][...]

# Flip the species code array vertically for each z-slice
speciescode_flipped = np.zeros((array_sz['z'], array_sz['y'], array_sz['x']), dtype='float32')
for zi in np.arange(0, array_sz['z']):
    speciescode_flipped[zi, :, :] = np.flipud((speciescode[zi, :, :]))

# Change speciescode to shape (ny, nx, nz) from (nz, ny, nx)
speciescode = np.transpose(speciescode_flipped, (1, 2, 0))


In [ ]:
# write to a new fuels-array.zip file
store = zarr.ZipStore(gui_dir + "fuels-array-new.zip", mode = 'w')

# write a dictionary for the attributes in fast fuels
ff_zarr_attrs = {'dx': 2.0,
                 'dy': 2.0,
                 'dz': 1.0,
                 'nx': nx,
                 'ny': ny,
                 'nz': nz,
                 'pad': 0,
                 'xmax': xmax,
                 'xmin': xmin,
                 'ymax': ymax,
                 'ymin': ymin}

# Create the root group and subgroups for canopy and surface
g1 = zarr.group(store)
canopy = g1.create_group('canopy')
surface = g1.create_group('surface')

# Canopy group: create datasets and assign data
d1 = canopy.create_dataset('FMC', shape=canopy_shape, dtype = np.float32)
d1[:,:,:] =  make_canopy_arr(fmc[:,:,:])
d2 = canopy.create_dataset('SAV', shape=canopy_shape, dtype = np.float32)
d2[:,:,:] =  make_canopy_arr(sav[:,:,:])
d3 = canopy.create_dataset('bulk-density', shape=canopy_shape, dtype = np.float32)
d3[:,:,:] =  make_canopy_arr(bulkdensity[:,:,:])
d4 = canopy.create_dataset('species-code', shape=canopy_shape, dtype = np.uint16)
d4[:,:,:] =  (speciescode[:,:,:])

# Surface group: create datasets and assign 2D surface data
d5 = surface.create_dataset('DEM', shape=surface_shape, dtype = np.float32)
d5 = dem[:,:,0]
d6 = surface.create_dataset('FMC', shape=surface_shape, dtype = np.float32)
d6 = fmc[:,:,0]
d7 = surface.create_dataset('SAV', shape=surface_shape, dtype = np.float32)
d7 = sav[:,:,0]
d8 = surface.create_dataset('bulk-density', shape=surface_shape,  dtype = np.float32)
d8 = bulkdensity[:,:,0]
d9 = surface.create_dataset('fuel-depth', shape=surface_shape, dtype = np.float32)
d9 = fueldepth[:,:,0]

# Update attributes for the root group
g1.attrs.update(ff_zarr_attrs)

# Close the store to finish writing
store.close()


In [ ]:
# rename new fuels-array zip file to fuels-array.zip
os.rename(gui_dir + "fuels-array-new.zip", gui_dir + "fuel-array.zip")